In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt
import Fvvsc

- (1a) load data from delaney's folder
- (1b) creat function:
    - input N x 4 matrix with all the density matrix info
    - output: N x N$_{\mathrm{big}}$ x N$_{\mathrm{big}}$ x 4 matrix that holds the components of F(p1, p2, p3)

In [2]:
#1a
data = np.load('Run35.npz', allow_pickle = True)
rawdata = data['raw'].item() #loads dictionary
rawdata.keys() # shows stored info

dict_keys(['initial array', 'time', 'dt', 'dN', 'nu3D', 'nubar3D', 'nu_init matrix', 'nu_bar_init matrix'])

In [3]:
nu3D = rawdata['nu3D'] #3D matrix N_time x N x 4
# N_time is the number of time steps so [0,:,:] is a matrix for the first time steps
nu3D

array([[[ 4.87502604e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00],
        [ 4.75020813e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00],
        [ 4.62570155e-01,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00],
        ...,
        [ 5.01721647e-05,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00],
        [ 4.77253561e-05,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00],
        [ 4.53978687e-05,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]],

       [[ 4.87502836e-01, -4.27087541e-02, -2.79853529e-02,
          9.98695088e-01],
        [ 4.75021265e-01, -3.79413304e-02, -1.43663969e-02,
          9.99175790e-01],
        [ 4.62570815e-01, -3.70438631e-02, -9.60371930e-03,
          9.99266153e-01],
        ...,
        [ 5.01768944e-05, -3.62791892e-02,  2.58726903e-03,
          9.99293030e-01],
        [ 4.77298779e-05, -3.62787153e-02,  2.60180908e-03,
          9.99292781e-01],
        [ 4.540

In [4]:
nu3D[0,0,:]

array([0.4875026, 0.       , 0.       , 1.       ])

In [67]:
#1b
zero_array = np.zeros(4) # for out of range energy values, but as global variable

In [134]:
def compute_F_matrix(density_matrix):
    B = 2
    Nbig = B * N
   
    # Initialize the output matrix with zeros
    F_matrix = np.zeros((N, Nbig, Nbig, 4), dtype=np.complex128)

    for i in range(N): # dimension i is N because p1 goes from 0 to emax
        for j in range(Nbig): # dimension j is Nbig because p2 goes from 0 to B*emax
            for k in range(Nbig): # dimension k is Nbig because p3 goes from 0 to B*emax
                
                l = i + j - k # because p4 = p1 + p2 - p3 (feynmann diagram)
    
                p0xyz_1 = density_matrix[i, :]
            
                if j < N:
                    p0xyz_2 = density_matrix[j, :]
                else: 
                    zero_array
                
                if k < N:
                    p0xyz_3 = density_matrix[k, :]
                else: 
                    zero_array
                
                if l < 0 or l >= N:
                # when i didnt do this and just had l<0 i had problems... so here ya go
                    p0xyz_4 = zero_array 
                else:
                    p0xyz_4 = density_matrix[l, :]
               
            # WAIT THIS IS BACKWARDS
            
                # get F components for this combination of energy values (is it energies?)
                F_matrix[i, j, k, :] = Fvvsc.Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)

    return F_matrix


In [135]:
N = 1 
emax = 10 
B = 2 
Nbig = B*N
#delta = something??? i dont get what im supposed to do with delta

density_matrix_rand = np.random.rand(N, 4) # N rows each have 4 random numbers
# but instead of random numbers its the energy values
# or do i use the random number generator i made bc need complex ones too

F_matrix = compute_F_matrix(density_matrix_rand)

F_matrix[0,:,:,:]
#F_matrix

array([[[0.46515899+0.j, 0.08023261+0.j, 0.07067739+0.j, 0.12228036+0.j],
        [0.44561793+0.j, 0.09337657+0.j, 0.08225598+0.j, 0.1423127 +0.j]],

       [[0.44561793+0.j, 0.09337657+0.j, 0.08225598+0.j, 0.1423127 +0.j],
        [0.46515899+0.j, 0.08023261+0.j, 0.07067739+0.j, 0.12228036+0.j]]])

- need to use actual density matrix input but still confused about where to get that... 
wait was this the intermediate step i was supposed to do

# QUESTION: 
was the intermediate step i was supposed to do make an Nx4 matrix with p1,p2,p3? but thats N x 3?

[[p1a, p2a, p3a, p4a]
p1b, p2b, p3b, p4b], 
[p1c, p2c, p3c, p4c],
....
[p1n, p2n, p3n, p4n]] 

like that? like n of them but then p1 is actually just which array it is and then p1,p2,p3 are the next three so actually p2, p3, p4?